In [2]:
from FiinQuantX import FiinSession


username = 'DSTC_20@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

fi = client.FiinIndicator()    

In [ ]:
import pandas as pd
import numpy as np

# Đọc dataset
df_3years = pd.read_csv("") # Đường dẫn file dataset 3 năm gần nhất
df_ticker_fa_filter = pd.read_csv("final.csv")


In [ ]:
good_ratings = ['A+', 'A', 'B', 'C']
df_good_ratings = df_ticker_fa_filter[df_ticker_fa_filter['FA_Rating'].isin(good_ratings)]
ticker_list = df_good_ratings['ticker'].tolist()
print(len(ticker_list))

450


In [5]:
df_3years_filtered = df_3years[df_3years['ticker'].isin(ticker_list)]


In [6]:
def calculate_all_indicators(group):
    """
    Hàm để tính toán tất cả các chỉ số TA cho một nhóm (một ticker).
    """
    # Sắp xếp theo thời gian để đảm bảo tính toán đúng
    group = group.sort_values('timestamp').reset_index(drop=True)
    
    try:
        ema20  = fi.ema(group['close'], 20)
        sma20  = fi.sma(group['close'], 20)        # dùng cho price cross
        sma50  = fi.sma(group['close'], 50)
        sma200 = fi.sma(group['close'], 200)

        group['px_gt_SMA50']     = (group['close'] > sma50).astype(int)
        group['EMA20_gt_SMA50']  = (ema20 > sma50).astype(int)

        group['px_cross_up_SMA20'] = ((group['close'] > sma20) & (group['close'].shift(1) <= sma20.shift(1))).astype(int)
        group['px_cross_dn_SMA20'] = ((group['close'] < sma20) & (group['close'].shift(1) >= sma20.shift(1))).astype(int)
        group['px_cross_up_EMA20'] = ((group['close'] > ema20) & (group['close'].shift(1) <= ema20.shift(1))).astype(int)
        group['px_cross_dn_EMA20'] = ((group['close'] < ema20) & (group['close'].shift(1) >= ema20.shift(1))).astype(int)

        group['SMA20_cross_up_SMA50']  = ((sma20 > sma50) & (sma20.shift(1) <= sma50.shift(1))).astype(int)
        group['GoldenCross']           = ((sma50 > sma200) & (sma50.shift(1) <= sma200.shift(1))).astype(int)
        group['DeathCross']            = ((sma50 < sma200) & (sma50.shift(1) >= sma200.shift(1))).astype(int)

        # ====== Momentum ======
        # MACD: giữ 1 continuous + 1 event + 1 state
        macd        = fi.macd(group['close'])
        macd_signal = fi.macd_signal(group['close'])
        group['macd']            = macd
        group['macd_cross_up']   = ((macd > macd_signal) & (macd.shift(1) <= macd_signal.shift(1))).astype(int)
        group['macd_above_zero'] = (macd > 0).astype(int)

        # RSI: thêm event-based (cross 30/70) để tránh trùng với các flag tĩnh
        rsi14 = fi.rsi(group['close'], 14)
        group['rsi14']         = rsi14                    # continuous cho ML
        group['RSI_cross_up30'] = ((rsi14 >= 30) & (rsi14.shift(1) < 30)).astype(int)
        group['RSI_cross_dn70'] = ((rsi14 <= 70) & (rsi14.shift(1) > 70)).astype(int)

        # Stochastic: giữ 1 tín hiệu khác biệt (cross K>D), bỏ rebound20 để tránh trùng ý với RSI_up30
        stoch_k = fi.stoch(group['high'], group['low'], group['close'], 14)
        stoch_d = fi.stoch_signal(group['high'], group['low'], group['close'], 14, 3)
        group['stoch_cross_up'] = ((stoch_k > stoch_d) & (stoch_k.shift(1) <= stoch_d.shift(1))).astype(int)

        # ====== Volatility (Bollinger) ======
        bb_u = fi.bollinger_hband(group['close'], 20, 2)
        bb_l = fi.bollinger_lband(group['close'], 20, 2)
        group['bb_break_upper'] = (group['close'] > bb_u).astype(int)
        group['bb_break_lower'] = (group['close'] < bb_l).astype(int)

        # ====== ADX & DI ======
        group['adx'] = fi.adx(group['high'], group['low'], group['close'], 14)
        di_pos       = fi.adx_pos(group['high'], group['low'], group['close'], 14)
        di_neg       = fi.adx_neg(group['high'], group['low'], group['close'], 14)
        group['DI_spread'] = di_pos - di_neg   # dương: up-trend; âm: down-trend

        # ====== PSAR ======
        psar = fi.psar(group['high'], group['low'], group['close'], step=0.02, max_step=0.2)
        group['px_gt_PSAR']   = (group['close'] > psar).astype(int)
        group['PSAR_flip_up'] = ((group['close'] > psar) & (group['close'].shift(1) <= psar.shift(1))).astype(int)

        # ====== Ichimoku (gọn) ======
        conv   = fi.ichimoku_conversion_line(group['high'], group['low'], group['close'])
        base   = fi.ichimoku_base_line(group['high'], group['low'], group['close'])
        span_a = fi.ichimoku_a(group['high'], group['low'], group['close'])
        span_b = fi.ichimoku_b(group['high'], group['low'], group['close'])
        group['CONV_gt_BASE'] = (conv > base).astype(int)
        cloud_top = np.maximum(span_a, span_b)
        cloud_bot = np.minimum(span_a, span_b)
        group['px_above_cloud'] = (group['close'] > cloud_top).astype(int)
        group['px_in_cloud']    = ((group['close'] <= cloud_top) & (group['close'] >= cloud_bot)).astype(int)

        # ====== Volume & Flow ======
        group['mfi'] = fi.mfi(group['high'], group['low'], group['close'], group['volume'], 14)
        vol_ma20     = group['volume'].rolling(20).mean()
        group['vol_surge']   = (group['volume'] / vol_ma20.replace(0, np.nan)).fillna(0.0)
        group['bu_sd_ratio'] = (group['bu'] / group['sd'].replace(0, np.nan)).fillna(0.0)
        foreign_net          = (group['fb'] - group['fs']).astype(float)
        group['foreign_scaled'] = (foreign_net / group['volume'].replace(0, np.nan)).fillna(0.0)

        # ====== Price Action ======
        group['ret_1d']  = group['close'].shift(-1) / group['close'] - 1.0
        group['ret_5d']  = group['close'].shift(-5) / group['close'] - 1.0
        group['ret_20d'] = group['close'].shift(-20) / group['close'] - 1.0
        group['gap_open'] = (group['open'] / group['close'].shift(1) - 1.0)

        chg = group['close'].diff()
        up  = (chg > 0).astype(int)
        dn  = (chg < 0).astype(int)
        group['streak_up'] = up.groupby((up != up.shift()).cumsum()).cumsum() * up
        group['streak_dn'] = dn.groupby((dn != dn.shift()).cumsum()).cumsum() * dn

        return group
    except Exception as e:
        print(f"⚠️ Lỗi ticker {group.get('ticker', ['?'])[0]}: {e}")
        return group

In [7]:
df_with_indicators = df_3years_filtered.groupby('ticker', group_keys=False).apply(calculate_all_indicators)

In [8]:
print("\n📊 Dữ liệu sau khi tính toán indicators:")
display(df_with_indicators.head())


📊 Dữ liệu sau khi tính toán indicators:


,ticker,timestamp,open,high,low,close,volume,bu,sd,fb,...,mfi,vol_surge,bu_sd_ratio,foreign_scaled,ret_1d,ret_5d,ret_20d,gap_open,streak_up,streak_dn
0,A32,2022-08-30 00:00,25098.504282,25098.504282,25098.504282,25098.504282,100.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.000000,-0.089062,-0.093750,NaN,0,0
1,A32,2022-08-31 00:00,25098.504282,25098.504282,25098.504282,25098.504282,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.000000,-0.087500,-0.093750,0.000000,0,0
2,A32,2022-09-05 00:00,25098.504282,25098.504282,25098.504282,25098.504282,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,-0.072906,-0.087500,-0.093750,0.000000,0,0
3,A32,2022-09-06 00:00,21647.459944,26510.295148,21647.459944,23268.666455,300.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.147099,0.011225,-0.022483,-0.137500,0,1
4,A32,2022-09-07 00:00,26588.727974,26745.593626,26588.727974,26691.474976,1600.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,-0.143428,-0.118451,-0.147836,0.142684,1,0


In [10]:
# Lưu kết quả vào file CSV
df_with_indicators.to_csv("3years_with_indicators.csv", index=False)

In [11]:
df_ma_nganh = pd.read_csv("Ma_Nganh.csv")

# 2. Đổi tên cột để ghép nối cho nhất quán ('Mã' -> 'ticker', 'Ngành' -> 'industry')
df_ma_nganh = df_ma_nganh.rename(columns={'Mã': 'ticker', 'Ngành': 'industry'})

In [12]:
df_final = pd.merge(df_with_indicators, df_ma_nganh, on='ticker', how='left')
df_final

,ticker,timestamp,open,high,low,close,volume,bu,sd,fb,...,vol_surge,bu_sd_ratio,foreign_scaled,ret_1d,ret_5d,ret_20d,gap_open,streak_up,streak_dn,industry
0,A32,2022-08-30 00:00,25098.504282,25098.504282,25098.504282,25098.504282,100.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,-0.089062,-0.093750,NaN,0,0,Tiêu dùng không thiết yếu
1,A32,2022-08-31 00:00,25098.504282,25098.504282,25098.504282,25098.504282,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,-0.087500,-0.093750,0.000000,0,0,Tiêu dùng không thiết yếu
2,A32,2022-09-05 00:00,25098.504282,25098.504282,25098.504282,25098.504282,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,-0.072906,-0.087500,-0.093750,0.000000,0,0,Tiêu dùng không thiết yếu
3,A32,2022-09-06 00:00,21647.459944,26510.295148,21647.459944,23268.666455,300.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.147099,0.011225,-0.022483,-0.137500,0,1,Tiêu dùng không thiết yếu
4,A32,2022-09-07 00:00,26588.727974,26745.593626,26588.727974,26691.474976,1600.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,-0.143428,-0.118451,-0.147836,0.142684,1,0,Tiêu dùng không thiết yếu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331165,XMP,2025-08-25 00:00,15400.000000,15400.000000,15300.000000,15383.000000,600.0,0.0,600.0,0.0,...,0.389610,0.0,0.0,0.001105,NaN,NaN,0.000000,0,1,Năng lượng
331166,XMP,2025-08-26 00:00,15400.000000,15400.000000,15400.000000,15400.000000,200.0,0.0,200.0,0.0,...,0.131148,0.0,0.0,0.000000,NaN,NaN,0.001105,1,0,Năng lượng
331167,XMP,2025-08-27 00:00,15400.000000,15400.000000,15400.000000,15400.000000,2300.0,0.0,2300.0,0.0,...,1.446541,0.0,0.0,0.001494,NaN,NaN,0.000000,0,0,Năng lượng
331168,XMP,2025-08-28 00:00,15400.000000,15500.000000,15400.000000,15423.000000,1300.0,0.0,1300.0,0.0,...,0.787879,0.0,0.0,-0.001491,NaN,NaN,0.000000,1,0,Năng lượng


In [13]:
def calculate_ta_score(row):
    score = 0
    score += row['px_gt_SMA50'] * 2
    score += row['EMA20_gt_SMA50'] * 2
    score += row['px_cross_up_SMA20'] * 3
    score += row['GoldenCross'] * 5
    score += row['macd_above_zero'] * 2
    score += row['macd_cross_up'] * 3
    score += (row['rsi14'] > 50) * 2
    score += row['RSI_cross_up30'] * 2
    score += row['stoch_cross_up'] * 2
    score += (row['adx'] > 25) * 2
    score += (row['DI_spread'] > 0) * 2
    score += row['px_gt_PSAR'] * 2
    score += row['PSAR_flip_up'] * 3
    score += row['CONV_gt_BASE'] * 2
    score += row['px_above_cloud'] * 3
    score += (row['mfi'] > 50) * 2
    score += (row['vol_surge'] > 1.5) * 2
    score += (row['bu_sd_ratio'] > 1.0) * 2
    score += (row['foreign_scaled'] > 0) * 2
    return score

# Lọc dữ liệu để lấy dòng mới nhất cho mỗi ticker
latest_data = df_final.sort_values('timestamp').groupby('ticker').tail(1).reset_index(drop=True)
latest_data['TA_Score'] = latest_data.apply(calculate_ta_score, axis=1)

# Lấy top 10 TA_Score theo từng ngành
top_stocks_per_industry = latest_data.sort_values(['industry', 'TA_Score'], ascending=[True, False]).groupby('industry').head(10)
top_stocks_per_industry = top_stocks_per_industry[['ticker', 'industry', 'TA_Score']]
top_stocks_per_industry = top_stocks_per_industry.sort_values(['industry', 'TA_Score'], ascending=[True, False]).reset_index(drop=True)
top_stocks_per_industry.to_csv("top_stocks_per_industry.csv", index=False)
print("\n🏆 Top stocks per industry based on TA Score:")
display(top_stocks_per_industry)


🏆 Top stocks per industry based on TA Score:


,ticker,industry,TA_Score
0,PNT,Bất động sản,26
1,VHM,Bất động sản,21
2,HPI,Bất động sản,21
3,IDC,Bất động sản,21
4,KBC,Bất động sản,21
...,...,...,...
98,MBB,Tài chính,23
99,OCB,Tài chính,23
100,TCB,Tài chính,21
101,SGB,Tài chính,21


In [14]:
# Filter df_final theo các ticker trong top_stocks_per_industry
filtered_final = df_final[df_final['ticker'].isin(top_stocks_per_industry['ticker'])]
filtered_final.to_csv("filtered_final.csv", index=False)